# Notebook for the training of the Teresa Robot
## Importing all necesary tools

In [1]:
from src.gym_envs.RobotEnv import RobotEnv # Training environment
import numpy as np
import roslibpy # API of ROS
from src.robots.Teresa import Teresa # This is the representation of Teresa Robot

## Creating the connection with ROS

In [2]:
HOST = 'localhost'
PORT = 9090

client = roslibpy.Ros(host=HOST, port=PORT)

client.run() # This run the main loop of ROS

2020-08-24 12:34:59,230     INFO: Connection to ROS MASTER ready.


## Creating representation of the robot and introducing it in the Gym Environment

In [4]:
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller) # Training Environment

env.reset() # Restarting the environment to the initial state

0

## Testing the environment

In [5]:
for i in range(20):
    env.step(np.random.randint(4))
    env.render()

2020-08-24 12:35:44,709     INFO: Received image seq=0
2020-08-24 12:35:44,779     INFO: Received image seq=1
2020-08-24 12:35:44,839     INFO: Received image seq=2
2020-08-24 12:35:44,926     INFO: Received image seq=3
2020-08-24 12:35:44,991     INFO: Received image seq=4
2020-08-24 12:35:45,061     INFO: Received image seq=5
2020-08-24 12:35:45,131     INFO: Received image seq=6
2020-08-24 12:35:45,215     INFO: Received image seq=7
2020-08-24 12:35:45,257     INFO: Received image seq=8
2020-08-24 12:35:45,314     INFO: Received image seq=9
2020-08-24 12:35:50,562     INFO: Received image seq=10
2020-08-24 12:35:50,655     INFO: Received image seq=11
2020-08-24 12:35:50,738     INFO: Received image seq=12
2020-08-24 12:35:50,814     INFO: Received image seq=13
2020-08-24 12:35:50,949     INFO: Received image seq=14
2020-08-24 12:35:51,048     INFO: Received image seq=15
2020-08-24 12:35:51,149     INFO: Received image seq=16
2020-08-24 12:35:51,227     INFO: Received image seq=17
20